In [4]:
import datetime as dt

import numpy as np
import pandas as pd

from wmfdata import spark

# Retention rate

In [52]:
query_retention_rate='''
-- only count non-bot user 
SELECT 
          date_format(1st_month.user_creation_date,'yyyy-MM-01') AS user_creation_cohort_month, 
          1st_month.has_growth_features, 
          --SUM(CASE WHEN 2nd_month.edits IS NOT NULL THEN 1 ELSE 0 END) , COUNT(1)
         ROUND(SUM(CASE WHEN 2nd_month.edits IS NOT NULL THEN 1 ELSE 0 END) /COUNT(1), 6)  AS retention_rate
FROM    (          SELECT   h1.event_user_text                           AS user_name, 
                            h1.event_user_id                             AS user_id, 
                            m1.hp_enabled                                AS has_growth_features,
                            substr(h1.event_user_creation_timestamp,1,10) AS user_creation_date, 
                            count(*)                                  AS edits 
                   FROM     wmf.mediawiki_history AS h1
                   INNER JOIN nettrom_growth.hp_variant_test2 AS m1 
                            ON m1.user_id= h1.event_user_id and m1.wiki_db= h1.wiki_db
                   WHERE    snapshot = "{SNAPSHOT}" 
                   AND      h1.wiki_db='ptwiki' 
                   AND      h1.event_entity = "revision" 
                   AND      h1.event_type = "create" 
                   AND      NOT h1.event_user_is_created_by_system 
                   AND      size(h1.event_user_is_bot_by) = 0 
                   AND      h1.event_user_creation_timestamp BETWEEN "{START_YYYY_MM_DD}" AND      "{END_YYYY_MM_DD}" 
                   AND      unix_timestamp(h1.event_timestamp, "yyyy-MM-dd HH:mm:ss.0") < 
                   (unix_timestamp(h1.event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) 
                   GROUP BY h1.event_user_text, 
                            h1.event_user_id, 
                            m1.hp_enabled,
                            h1.event_user_creation_timestamp
                            ) 1st_month 
LEFT JOIN 
          ( 
                   SELECT   h2.event_user_text                           AS user_name, 
                            h2.event_user_id                             AS user_id, 
                            substr(h2.event_user_creation_timestamp,1,10) AS user_creation_date, 
                            count(*)                                  AS edits 
                   FROM     wmf.mediawiki_history AS h2
                   INNER JOIN nettrom_growth.hp_variant_test2 AS m2 
                            ON m2.user_id= h2.event_user_id and m2.wiki_db= h2.wiki_db
                   WHERE    h2.snapshot = "{SNAPSHOT}" 
                   AND      h2.wiki_db='ptwiki' 
                   AND      h2.event_entity = "revision" 
                   AND      h2.event_type = "create" 
                   AND      NOT h2.event_user_is_created_by_system 
                   AND      size(h2.event_user_is_bot_by) = 0 
                   AND      h2.event_user_creation_timestamp BETWEEN "{START_YYYY_MM_DD}" AND      "{END_YYYY_MM_DD}" 
                   AND      unix_timestamp(h2.event_timestamp, "yyyy-MM-dd HH:mm:ss.0") >= (unix_timestamp(h2.event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) 
                   AND      unix_timestamp(h2.event_timestamp, "yyyy-MM-dd HH:mm:ss.0") < (unix_timestamp(h2.event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.0")  + (60*24*60*60)) 
                   GROUP BY h2.event_user_text, 
                            h2.event_user_id, 
                            h2.event_user_creation_timestamp
                            ) 2nd_month 
ON        ( 
                    1st_month.user_id = 2nd_month.user_id 
          AND       1st_month.user_creation_date = 2nd_month.user_creation_date)
GROUP BY date_format(1st_month.user_creation_date,'yyyy-MM-01'), 1st_month.has_growth_features
ORDER BY user_creation_cohort_month
LIMIT 100000
'''

In [54]:
df_retention_rate = spark.run(query_retention_rate.format(
    SNAPSHOT = '2021-05',
    START_YYYY_MM_DD = '2020-10-01',
    END_YYYY_MM_DD = '2021-05-31'
))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [55]:
df_retention_rate

,user_creation_cohort_month,has_growth_features,retention_rate
0,2020-10-01,1,0.062640
1,2020-10-01,0,0.043103
2,2020-11-01,0,0.048943
3,2020-11-01,1,0.051081


# Net content edits

In [23]:
query_net_content_edits='''

WITH t1 AS
(
SELECT date_format(event_timestamp,'yyyy-MM-01') AS y_m,
    m1.hp_enabled AS has_growth_features,
    count(revision_id) AS non_reverted_edits
FROM wmf.mediawiki_history AS h
INNER JOIN nettrom_growth.hp_variant_test2 AS m1 ON m1.user_id= event_user_id and m1.wiki_db= h.wiki_db
WHERE snapshot= '{SNAPSHOT}' AND h.wiki_db='ptwiki'
    AND page_namespace_is_content
    -- for bots size(event_user_is_bot_by) returns 1/2, IP editors return -1, registered non-bot editors return 0
    AND  size(event_user_is_bot_by) <= 0 
    AND substr(event_timestamp,1,10) BETWEEN '{START_YYYY_MM_DD}' AND '{END_YYYY_MM_DD}'
    AND event_entity = "revision" AND (NOT revision_is_identity_reverted  OR revision_seconds_to_identity_revert > 172800)
GROUP BY date_format(event_timestamp,'yyyy-MM-01'), m1.hp_enabled
ORDER BY y_m
LIMIT 1000000
),
t2 AS
(
SELECT date_format(h1.event_timestamp,'yyyy-MM-01') AS y_m,
    m2.hp_enabled AS has_growth_features,
    count(distinct h1.revision_id) AS revert_edits
FROM wmf.mediawiki_history AS h1 
INNER JOIN nettrom_growth.hp_variant_test2 AS m2 ON m2.user_id= h1.event_user_id and m2.wiki_db= h1.wiki_db
LEFT JOIN wmf.mediawiki_history AS h2  -- h1 is revert edits, h2 is the edits being reverted
    ON h1.revision_id=h2.revision_first_identity_reverting_revision_id  
    AND h1.wiki_db = h2.wiki_db
    AND h1.snapshot= h2.snapshot 
WHERE h1.wiki_db = 'ptwiki' AND h1.snapshot='{SNAPSHOT}'
    AND h1.page_namespace_is_content
    -- for bots size(event_user_is_bot_by) returns 1 or 2, IP editors return -1, registered non-bot editors return 0
    AND size(h1.event_user_is_bot_by) <= 0 AND size(h2.event_user_is_bot_by) <= 0
    AND substr(h1.event_timestamp,1,10) BETWEEN '{START_YYYY_MM_DD}' AND '{END_YYYY_MM_DD}'
    AND h1.event_entity = "revision" and h2.event_entity = "revision" 
    AND (NOT h1.revision_is_identity_reverted  OR h1.revision_seconds_to_identity_revert > 172800)
    AND h2.revision_is_identity_reverted AND h2.revision_seconds_to_identity_revert < 172800
    AND h1.revision_is_identity_revert
    AND h1.event_entity = 'revision' and  h2.event_entity = 'revision'
GROUP BY date_format(h1.event_timestamp,'yyyy-MM-01'), m2.hp_enabled
ORDER BY y_m
LIMIT 10000
)
SELECT t1.y_m, t1.has_growth_features, t1.non_reverted_edits - COALESCE(t2.revert_edits,0) AS net_non_reverted_edits
FROM  t1
LEFT JOIN t2 ON t1.y_m=t2.y_m and t1.has_growth_features=t2.has_growth_features
ORDER BY t1.y_m, t1.has_growth_features

'''

In [24]:
df_net_content_edits = spark.run(query_net_content_edits.format(
    SNAPSHOT = '2021-05',
    START_YYYY_MM_DD = '2020-10-01',
    END_YYYY_MM_DD = '2021-05-31'
))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [25]:
df_net_content_edits

,y_m,has_growth_features,net_non_reverted_edits
0,2020-10-01,0,144
1,2020-10-01,1,584
2,2020-11-01,0,1699
3,2020-11-01,1,7119
4,2020-12-01,0,460
5,2020-12-01,1,1630
6,2021-01-01,0,167
7,2021-01-01,1,1376
8,2021-02-01,0,222
9,2021-02-01,1,809
